In [447]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

from datetime import timedelta, date, datetime
from dateutil.relativedelta import relativedelta

import sys

from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

Import Datasets

In [448]:
rating_df = pd.read_csv("../data/custom/final_ratings.csv")
games_df = pd.read_csv("../data/custom/game_learn_df.csv")
games_df.rename(columns={'BGGId':'bgg_id'},inplace=True)
rating_df.rename(columns={'BGGId':'bgg_id'},inplace=True)

### Predefined Prep Functions

In [449]:
def UserFrame(rating, games, UName):
    u_rating = rating[rating['Username'] == UName]
    combined_df = u_rating.merge(games, on='bgg_id')
    return(combined_df)

In [450]:
def minbo_reduction(minbo, wf, game_id):
    filtered_df = wf[wf['bgg_id'] == game_id]
    visitors = filtered_df['Username'].unique()
    filtered_df = wf[wf['Username'].isin(visitors)]
    count_table = filtered_df['bgg_id'].value_counts().reset_index()
    count_table.columns = ['bgg_id', 'count']
    # print(count_table['count'].describe())
    if ((count_table['count'].describe().iloc[6]) < minbo):
        print(f'minbo to large. new mibo set to {count_table["count"].describe().iloc[6]}')
        minbo = (count_table["count"].describe().iloc[6])
    count_table=count_table.loc[count_table['count']>=minbo,'bgg_id']
    return(count_table)

In [451]:
keep_users = set(
    (
        (rating_df.groupby(
            'Username'
        )[['bgg_id']].count()
        ).loc[(
            rating_df.groupby(
                'Username'
            )[['bgg_id']].count()
        )['bgg_id']>12].reset_index()
    )['Username']
)

In [452]:
redu_rating = rating_df.loc[rating_df['Username'].isin(keep_users)]

In [453]:
red_game = games_df.drop(columns=(['min_age', 'min_time']))

#Remove Multiple Wargames
drop_war = list(red_game.filter(regex='War').columns)
drop_war.remove('War Game')
red_game.drop(columns=(drop_war), inplace=True)

#Remove multiple Worker Placement
drop_worker = list(red_game.filter(regex='Worker').columns)
drop_worker.remove('Worker Placement')
red_game.drop(columns=(drop_worker), inplace=True)

#Remove multiple Dice
drop_dice = list(red_game.filter(regex='Dice').columns)
drop_dice.remove('Dice')
red_game.drop(columns=(drop_dice), inplace=True)

#Remove multiple Card
drop_card = list(red_game.filter(regex='Card').columns)
drop_card.remove('Card Game')
red_game.drop(columns=(drop_card), inplace=True)

#Remove multiple Animal
drop_animal = list(red_game.filter(regex='Animal').columns)
drop_animal.remove('Animals')
red_game.drop(columns=(drop_animal), inplace=True)


# red_game.columns[75:]
# list(red_game.filter(regex='Village').columns)

#### Scaling Columns

In [454]:
red_game.set_index('bgg_id',inplace=True)
# red_game.head(1)

In [455]:
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

scaler = MinMaxScaler()
for col in red_game.columns:
    colly = red_game.loc[:, col].values.reshape(-1, 1)
    red_game.loc[:, col] = scaler.fit_transform(colly)

# Print the scaled DataFrame
print("Scaled Dataset Using MinMaxScaler")
red_game['avg_rating'] = red_game['avg_rating']#*2
red_game['complexity']  =   red_game['complexity']# *2
# red_game

Scaled Dataset Using MinMaxScaler


In [456]:
red_game.reset_index(inplace=True)

In [457]:
red_game=red_game.dropna(axis='columns')

In [458]:
## Prep Prediciton
def weight_game(user='ecoboardgeek123', wd=rating_df):
    wd = wd.loc[wd['Username']==user]
    mean_rating = wd['Rating'].mean()
    wd['Rating']=wd['Rating']-mean_rating
    result_dict = wd.set_index('bgg_id')['Rating'].to_dict()
    return(result_dict)

test_user = weight_game()

In [459]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def similar_games(games_df=red_game, alt=10, bgg_ids_with_weights={}):
    # Get the attributes of the specified games in the input list
    selected_games_attributes = games_df[games_df['bgg_id'].isin(bgg_ids_with_weights.keys())].iloc[:, 2:]

    if selected_games_attributes.empty:
        print("No games found for the specified bgg_ids.")
        return pd.DataFrame()

    # Calculate the similarity between all games and the specified games using cosine similarity
    similarity_scores = (cosine_similarity(selected_games_attributes, games_df.iloc[:, 2:]))

    # Apply the weights to the similarity scores
    for bgg_id, weight in bgg_ids_with_weights.items():
        indices = games_df[games_df['bgg_id'] == bgg_id].index
        if len(indices) > 0:
            index = indices[0]
            similarity_scores[:, index] *= weight

    # Sum the similarity scores across the rows
    total_similarity_scores = (similarity_scores.mean(axis=0)
)
    # Add similarity scores as a new column to the DataFrame
    games_with_similarity = games_df.assign(similarity=total_similarity_scores)

    #Get rid of Games in rated Frame
    # print('bla',games_with_similarity.loc[~games_with_similarity['bgg_id'].isin(list(bgg_ids_with_weights.keys()))])#.loc[games_with_similarity['bgg_id'].isin(list[bgg_ids_with_weights.keys()])], 'test')
    games_with_similarity=games_with_similarity.loc[~games_with_similarity['bgg_id'].isin(list(bgg_ids_with_weights.keys()))]
    
    # Sort the games based on similarity scores in descending order
    top_similar_games = games_with_similarity.sort_values('similarity', ascending=False).head(alt)

    return top_similar_games[['bgg_id', 'similarity']]

In [460]:
similar_games(games_df=red_game, bgg_ids_with_weights=weight_game(), alt=10000)

,bgg_id,similarity
20919,176565,0.268753
30565,298155,0.268753
28989,277410,0.268753
35547,381639,0.268753
28129,265980,0.268753
...,...,...
8676,19026,0.123730
35370,374043,0.123721
19636,162443,0.123712
20842,175763,0.123704
